In [1]:
import torch
import numpy as np
from tqdm import tqdm
from scipy.stats.distributions import chi2
from scipy.stats import pearsonr
import pickle

import matplotlib.pyplot as plt
from scipy.stats import linregress
import seaborn as sns
import random
import matplotlib.animation as animation
import sys, os

from utils.plots import *
from utils.funcs import *


from boltzmann_machines.RTRBM import RTRBM
from boltzmann_machines.RTRBM_no_bias import RTRBM_nobias
from boltzmann_machines.RBM import RBM

from data.load_data import *
from data.mock_data import *
from data.reshape_data import *

In [ ]:
# Parameters:

N_V = 20
T = 30
N_H = 3
device = 'cpu'
n_rtrbms = 1
n_batches = 500
n_connections = 1

weights = []
generated = []
trained = []

for _ in range(n_rtrbms):
    U = 2 * np.random.rand(N_H, N_H) - 1
    W = 2 * torch.rand(N_H, N_V) - 1
    sp = 1 - n_connections / N_H

    for i, idx in enumerate(torch.randperm(N_V)):
        if torch.sum(W[:, idx] == 0) / N_H < sp:
            W[:, idx][np.random.permutation(N_H)[:int(np.ceil(sp * N_H))]] = 0

    b_init = torch.zeros(1, N_H, dtype=torch.float, device=device)
    b_V = torch.zeros(1, N_V, dtype=torch.float, device=device)
    b_H = torch.zeros(1, N_H, dtype=torch.float, device=device)

    data = torch.zeros(N_V, T, dtype=torch.float)

    rtrbm = RTRBM(data, N_H=N_H, device='cpu')

    rtrbm.W = torch.tensor(W, device=device, dtype=torch.float)
    rtrbm.U = torch.tensor(U, device=device, dtype=torch.float)
    rtrbm.b_init = b_init
    rtrbm.b_V = b_V
    rtrbm.b_H = b_H

    data = torch.zeros(N_V, T, n_batches)
    rt = torch.zeros(N_H, T, n_batches)
    for batch in tqdm(range(n_batches)):
        v_start = (torch.rand(N_V) > 0.2)*1.0
        data[:, :, batch], rt[:, :, batch] = rtrbm.sample(v_start.type(torch.float), chain=T, pre_gibbs_k=100, gibbs_k=100, mode=1, disable_tqdm=True)

    for sp in [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]:

        rtrbm_nobias = RTRBM_nobias(data, N_H=N_H, device='cpu')

        for _ in range(2000):
            rtrbm_nobias.learn(n_epochs=1, lr=1e-4, start_decay=200, batchsize=100, mom=0, wc=0, sp=sp, x=2)

            W = rtrbm_nobias.W
            U = rtrbm_nobias.U
            weights += [[W], [U]]

        generated += [rtrbm]
        trained += [rtrbm_nobias]


C:\Users\Casper\AppData\Local\Temp/ipykernel_16276/3564687833.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rtrbm.W = torch.tensor(W, device=device, dtype=torch.float)
  0%|          | 0/1 [00:00<?, ?it/s]

In [63]:
torch.save(generated, 'generated1.pt')
torch.save(trained, 'trained1.pt')